## Import dependencies

In [ ]:
import random
import copy
import logging

# Get the root logger and set its level to DEBUG
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Set up classes

In [ ]:
class Squad:
    def __init__(self, name, *unit_pairs):
        self.name = name
        self.unit_pairs = unit_pairs
        self.units = []
        self.spawn_squad()

    def spawn_squad(self):
        self.units = []
        for unit, count in self.unit_pairs:
            self.units.extend([copy.deepcopy(unit) for _ in range(count)])
    
    def reset_squad_health(self):
        for unit in self.units:
            unit.health = unit.max_health
    
    def is_alive(self):
        return any(unit.health > 0 for unit in self.units)

    def get_alive_unit(self):
        for unit in self.units:
            if unit.is_alive():
                return unit
        return None
    
    def total_health(self):
        return sum(unit.health for unit in self.units)

class Weapon:
    def __init__(self, name, attacks, ballistic_skill, strength, armor_piercing, damage):
        self.name = name
        self.attacks = attacks
        self.ballistic_skill = ballistic_skill
        self.strength = strength
        self.armor_piercing = armor_piercing
        self.damage = damage

    def get_attacks(self):
        if callable(self.attacks):
            logging.debug(f"The method is: {self.get_attacks.__name__}. The amount of attacks was: {self.attacks()}")
            return self.attacks()
        else:
            return self.attacks
        
    def get_damage(self):
        if callable(self.damage):
            logging.debug(f"The method is: {self.get_damage.__name__}. The amount of damage was: {self.damage()}")

            return self.damage()
        else:
            return self.damage

class Unit:
    def __init__(self, name, weapons, toughness, armor, max_health, feel_no_pain=None, invulnerable_save=None,):
        self.name = name
        self.weapons = weapons
        self.toughness = toughness
        self.armor = armor
        self.invulnerable_save = invulnerable_save
        self.max_health = max_health
        self.health = max_health
        self.feel_no_pain = feel_no_pain

    def roll_dice(self):
        return random.randint(1, 6)

    def does_it_hit(self, weapon):
        dice_roll = self.roll_dice()
        outcome = dice_roll >= weapon.ballistic_skill
        logging.debug(f"The method is: {self.does_it_hit.__name__}. Weapon: {weapon.name}, Dice Roll: {str(dice_roll)}, Ballistic Skill: {weapon.ballistic_skill}, Outcome: {outcome}")
        return dice_roll >= weapon.ballistic_skill

    def does_it_wound(self, opponent_toughness, weapon):
        roll = self.roll_dice()
        outcome = False
        if weapon.strength == opponent_toughness and roll >= 4:
            outcome = True
        elif weapon.strength > opponent_toughness and roll >= 3:
            outcome = True
        elif weapon.strength >= 2 * opponent_toughness and roll >= 2:
            outcome = True
        elif weapon.strength < opponent_toughness and roll >= 5:
            outcome = True
        elif weapon.strength <= 0.5 * opponent_toughness and roll == 6:
            outcome = True

        logging.debug(f"The method is: {self.does_it_wound.__name__}. Weapon: {weapon.name}, Dice Roll: {str(roll)}, Opponent Toughness: {opponent_toughness}, Outcome: {outcome}")
        return outcome

    def does_the_armor_save(self, weapon):
        if self.invulnerable_save:
            modified_armor = min(self.invulnerable_save, self.armor - weapon.armor_piercing)
        else:
            modified_armor=self.armor - weapon.armor_piercing
        dice_roll = self.roll_dice()
        outcome = dice_roll < modified_armor
        logging.debug(f"The method is: {self.does_the_armor_save.__name__}, Weapon: {weapon.name}, Dice Roll: {str(dice_roll)}, Modified Armor: {modified_armor}, Outcome: {outcome}")
        return outcome

    def feel_no_pain_save(self,damage):
        if self.feel_no_pain:
            remaining_damage=0
            for _ in range(damage):
                dice_roll = self.roll_dice()
                if dice_roll < self.feel_no_pain:
                    remaining_damage += 1
            logging.debug(f"The method is: {self.feel_no_pain_save.__name__}, Unit: {self.name},Incoming damage: {damage}, Remaining damage: {remaining_damage}")
        else:
            remaining_damage=damage
            logging.debug(f"The method is: {self.feel_no_pain_save.__name__}, No feel no pain.,Incoming damage: {damage}, Remaining damage: {remaining_damage}")
        return remaining_damage
    
    def is_alive(self):
        return self.health > 0

In [ ]:
# unit_pairs=[(assault_intercessor_ranged, 2)]

# testunits = []
# for unit, count in unit_pairs:
#     # testunits.extend([copy.deepcopy(unit) for _ in range(count)])
#     testunits.extend([unit for _ in range(count)])
#     testunits[0].health = 99
# testunits

## Set up Weapons, Units and Squads

In [ ]:
# Variable number of attacks
def roll_dice_plus_variable(dice_number,added_variable):
    return random.randint(1, dice_number) + added_variable

# Create weapons
bolter = Weapon(name='bolter', attacks=2, ballistic_skill=3, strength=4, armor_piercing=0, damage=2)
genestealer_claws = Weapon(name='genestealer_claws',attacks=4, ballistic_skill=2, strength=4, armor_piercing=-2, damage=1)
heavy_bolt_pistol = Weapon(name='heavy_bolt_pistol', attacks=1, ballistic_skill=3, strength=4, armor_piercing=-1, damage=1)
astartes_chainsword = Weapon(name='astartes_chainsword', attacks=4, ballistic_skill=3, strength=4, armor_piercing=-1, damage=1)
old_one_eye_strike = Weapon(name='old_one_eye_strike', attacks=6, ballistic_skill=3, strength=14, armor_piercing=-3, damage=lambda: roll_dice_plus_variable(6,1))
old_one_eye_sweep = Weapon(name='old_one_eye_sweep', attacks=12, ballistic_skill=3, strength=6, armor_piercing=-1, damage=1)
macro_plasma_incinerator = Weapon(name='macro_plasma_incinerator', attacks=lambda: roll_dice_plus_variable(6,1), ballistic_skill=3, strength=8, armor_piercing=-3, damage=2)
macro_plasma_incinerator_supercharged = Weapon(name='macro_plasma_incinerator', attacks=lambda: roll_dice_plus_variable(6,1), ballistic_skill=3, strength=9, armor_piercing=-4, damage=3)
storm_bolter = Weapon(name='storm_bolter', attacks=4, ballistic_skill=3, strength=4, armor_piercing=0, damage=1)
onslaught_gatling_gun = Weapon(name='onslaught_gatling_gun', attacks=8, ballistic_skill=3, strength=5, armor_piercing=0, damage=1)




# Create units
assault_intercessor_ranged = Unit(name='assault_intercessor_ranged', weapons=[heavy_bolt_pistol], toughness=4, armor=3,  max_health=2)
assault_intercessor_melee = Unit(name='assault_intercessor_melee', weapons=[heavy_bolt_pistol,astartes_chainsword], toughness=4, armor=3,  max_health=2)
genestealer = Unit(name='Genestealer', weapons=[genestealer_claws], toughness=4, armor=5,  max_health=2, invulnerable_save=5)
old_one_eye=Unit(name='old_one_eye', weapons=[old_one_eye_strike,old_one_eye_sweep], toughness=9, armor=2,  max_health=9, feel_no_pain=5)
redemptor_dreadnought_ranged=Unit(name='redemptor_dreadnought_ranged', weapons=[macro_plasma_incinerator,storm_bolter,onslaught_gatling_gun], toughness=10, armor=2,  max_health=12)
redemptor_dreadnought_ranged_supercharged=Unit(name='redemptor_dreadnought_ranged', weapons=[macro_plasma_incinerator_supercharged,storm_bolter,onslaught_gatling_gun], toughness=10, armor=2,  max_health=12)


assault_intercessor_ranged_squad = Squad('Assault Intercessor Ranged Squad', (assault_intercessor_ranged, 5))
assault_intercessor_melee_squad = Squad('Assault Intercessor Melee Squad', (assault_intercessor_melee, 5))
genestealer_squad = Squad('Genestealer Squad', (genestealer, 10))
old_one_eye_squad = Squad('Old One Eye Squad', (old_one_eye, 1))  # Assuming you want only one Old One Eye in the squad
redemptor_dreadnought_ranged_squad = Squad('Redemptor Dreadnought Ranged Squad', (redemptor_dreadnought_ranged, 1))
redemptor_dreadnought_ranged_supercharged_squad = Squad('Redemptor Dreadnought Ranged Supercharged Squad', (redemptor_dreadnought_ranged_supercharged, 1))

## Set up simulation

In [ ]:
class Simulation:
    def __init__(self, attacker_squad, defender_squad, trials=5):
        attacker_squad.spawn_squad()
        defender_squad.spawn_squad()
        self.attacker_squad = attacker_squad
        self.defender_squad = defender_squad
        self.trials = trials
        self.initial_defender_squad_health=defender_squad.total_health()
        self.initial_number_of_defender_units=len(defender_squad.units)

        

    def run(self):

        total_damage = 0
        total_health_reduced = 0
        total_deaths = 0
        for _ in range(self.trials):
            logging.debug("\n\n\nStarting a new trial")
            self.attacker_squad.reset_squad_health()
            self.defender_squad.reset_squad_health()
            for attacker in self.attacker_squad.units:
                for weapon in attacker.weapons:
                    for _ in range(weapon.get_attacks()):
                        if attacker.does_it_hit(weapon):
                            defender = self.defender_squad.get_alive_unit()
                            if defender is None:
                                logging.debug("All units in the squad are dead")
                                break
                            if attacker.does_it_wound(defender.toughness, weapon):
                                if defender.does_the_armor_save(weapon):
                                    damage = weapon.get_damage()
                                    remaining_damage=defender.feel_no_pain_save(damage)
                                    logging.debug(f"Unit: {defender.name}, Health: {defender.health}, Damage: {remaining_damage}")
                                    total_damage += remaining_damage
                                    for _ in range(remaining_damage):
                                        defender.health -= 1
                                        total_health_reduced += 1
                                        logging.debug(f"Reduced one health. total health reduced is: {total_health_reduced}")

                                        if defender.health <= 0:
                                            total_deaths += 1
                                            logging.debug(f"Unit: {defender.name} has died : ( Health: {defender.health}, Damage: {remaining_damage}")
                                            break
        average_health_reduced=total_health_reduced / self.trials
        average_damage=total_damage / self.trials
        average_deaths=total_deaths / self.trials
        percentage_of_defender_squad_health_reduced=round(average_health_reduced / self.initial_defender_squad_health,3)
        overkill_percentage=round(average_damage / self.initial_defender_squad_health,3)
        return average_damage,average_health_reduced,percentage_of_defender_squad_health_reduced,overkill_percentage,self.initial_number_of_defender_units,average_deaths



## Performance profiling

In [ ]:
import cProfile
import pstats

logger.setLevel(logging.INFO)
number_of_trials=10000
def run_many_simulations():
    # for _ in range(number_of_trials):
    simulation = Simulation(redemptor_dreadnought_ranged_squad, old_one_eye_squad, trials=number_of_trials)
    result = simulation.run()

profiler = cProfile.Profile()
profiler.enable()
run_many_simulations()
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()

# Get the total time
total_time = sum([x.totaltime for x in profiler.getstats()])
logger.info(f'Average time per simulation: {round(total_time/number_of_trials,4)}')

## Run comparisons

In [ ]:
import pandas as pd

# Create squads


# Create a list of all squads
# squads = [assault_intercessor_ranged_squad, assault_intercessor_melee_squad, genestealer_squad,old_one_eye_squad,redemptor_dreadnought_ranged_squad,redemptor_dreadnought_ranged_supercharged_squad]
squads = [old_one_eye_squad,redemptor_dreadnought_ranged_squad,redemptor_dreadnought_ranged_supercharged_squad,genestealer_squad]

# Create an empty list to store the results
results = []

trials=1000
logger.info("Total number of trials: " + str(trials*len(squads)*len(squads)))

# Iterate over all pairs of squads
for i in range(len(squads)):
    for j in range(len(squads)):
        # Run a simulation for the current pair of squads
        simulation = Simulation(squads[i], squads[j], trials=1000)
        result = simulation.run()
        # Unpack the result list
        average_damage, average_health_reduced, percentage_of_defender_squad_health_reduced, overkill_percentage, initial_number_of_defender_units, average_deaths = result

        # Store the result in the results list
        results.append({
            'attacker': squads[i].name,
            'defender': squads[j].name,
            'average_damage': average_damage,
            'average_health_reduced': average_health_reduced,
            'percentage_of_defender_squad_health_reduced': percentage_of_defender_squad_health_reduced,
            'overkill_percentage': overkill_percentage,
            'initial_number_of_defender_units': initial_number_of_defender_units,
            'average_deaths': average_deaths
        })

# Convert the results list to a DataFrame
df = pd.DataFrame(results)
df